1.Variables i = faculty number (1-12) 
2.j = course number k = trimester (1, 2, 3) 
3.Pi = Total preferences of faculty i 
4.Ci = Total credits of faculty i for each trimester 
5.Ti = Total credits of faculty i for whole academic year 
6.Gj = total groups for each course

O.F Max Z = ΣPi

Constraints

total credits for each faculty in each trimester
trimester credit 0 <= ΣC <= 6

total credits for each faculty in whole academic year
year credit 10 <= ΣT <= 16

total groups for each course
course groups min <= ΣG <= max

In [1]:
pip install pulp


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install openpyxl


[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import random
from pulp import *

In [5]:
df = pd.read_excel("C:/Users/cleme/OneDrive/桌面/doreen/data_for_the_case.xlsx", engine="openpyxl")
df.head()

,Course Code,Course Name,Credits,Trimester,Min Nb Groups,Max Nb Groups,Faculty 1,Faculty 2,Faculty 3,Faculty 4,Faculty 5,Faculty 6,Faculty 7,Faculty 8,Faculty 9,Faculty 10,Faculty 11,Faculty 12
0,T101,Operations Management,3,1,1,1,0,5,2,0,5,4,3,0,3,5,2,2
1,T102,Data-Driven Decision Making,2,1,1,3,1,5,4,5,5,1,4,0,0,5,0,5
2,T103,Service Operations,2,1,1,1,1,0,4,1,3,0,3,1,3,4,2,5
3,T104,Project Management,2,1,2,2,2,2,0,0,1,4,0,5,2,1,1,5
4,T105,Simulation and System Dynamics,2,1,3,3,4,3,0,4,3,1,0,0,2,5,1,0


In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import pandas as pd

df = pd.read_excel("C:/Users/cleme/OneDrive/桌面/doreen/data_for_the_case.xlsx")
courses = df["Course Code"].tolist()
faculty = [col for col in df.columns if "Faculty" in col]
trimesters = [1, 2, 3]

model = LpProblem("Course_Allocation", LpMaximize)

# 建立決策變數 x[i,j,k]
x = {(i, j, k): LpVariable(f"x_{i}_{j}_{k}", cat="Binary") 
     for i in faculty for j in courses for k in trimesters}

# 建立目標函數：Maximize preference sum
model += lpSum(df.loc[df["Course Code"] == j, i].values[0] * x[i, j, k]
               for i in faculty for j in courses for k in trimesters)

# 限制條件 1: 每位老師每個 trimester 最多 6 學分
for i in faculty:
    for k in trimesters:
        model += lpSum(df.loc[df["Course Code"] == j, "Credits"].values[0] * x[i, j, k]
                       for j in courses if df.loc[df["Course Code"] == j, "Trimester"].values[0] == k) <= 6

# 限制條件 2: 每位老師全年總學分在 10~16
for i in faculty:
    model += lpSum(df.loc[df["Course Code"] == j, "Credits"].values[0] * x[i, j, k]
                   for j in courses for k in trimesters if df.loc[df["Course Code"] == j, "Trimester"].values[0] == k) >= 10
    model += lpSum(df.loc[df["Course Code"] == j, "Credits"].values[0] * x[i, j, k]
                   for j in courses for k in trimesters if df.loc[df["Course Code"] == j, "Trimester"].values[0] == k) <= 16

# 限制條件 3: 每門課只能在該 trimester 被安排，且符合 min/max group 限制
for _, row in df.iterrows():
    j = row["Course Code"]
    k = row["Trimester"]
    min_g = row["Min Nb Groups"]
    max_g = row["Max Nb Groups"]
    model += lpSum(x[i, j, k] for i in faculty) >= min_g
    model += lpSum(x[i, j, k] for i in faculty) <= max_g
    # 不允許該課程被排進錯誤 trimester
    for other_k in trimesters:
        if other_k != k:
            model += lpSum(x[i, j, other_k] for i in faculty) == 0


In [7]:
model.solve()

# 輸出結果
for var in model.variables():
    if var.varValue == 1:
        print(var.name)
print("Total Happiness Index:", model.objective.value())


x_Faculty_10_T102_1
x_Faculty_10_T105_1
x_Faculty_10_T212_2
x_Faculty_10_T214_2
x_Faculty_10_T301_3
x_Faculty_10_T306_3
x_Faculty_10_T311_3
x_Faculty_11_T107_1
x_Faculty_11_T109_1
x_Faculty_11_T112_1
x_Faculty_11_T203_2
x_Faculty_11_T209_2
x_Faculty_11_T312_3
x_Faculty_12_T103_1
x_Faculty_12_T104_1
x_Faculty_12_T205_2
x_Faculty_12_T213_2
x_Faculty_12_T307_3
x_Faculty_12_T311_3
x_Faculty_12_T314_3
x_Faculty_1_T105_1
x_Faculty_1_T112_1
x_Faculty_1_T201_2
x_Faculty_1_T211_2
x_Faculty_1_T308_3
x_Faculty_1_T309_3
x_Faculty_2_T101_1
x_Faculty_2_T110_1
x_Faculty_2_T203_2
x_Faculty_2_T210_2
x_Faculty_2_T301_3
x_Faculty_2_T302_3
x_Faculty_2_T314_3
x_Faculty_3_T111_1
x_Faculty_3_T206_2
x_Faculty_3_T207_2
x_Faculty_3_T305_3
x_Faculty_3_T306_3
x_Faculty_4_T108_1
x_Faculty_4_T109_1
x_Faculty_4_T112_1
x_Faculty_4_T204_2
x_Faculty_4_T302_3
x_Faculty_4_T303_3
x_Faculty_4_T305_3
x_Faculty_5_T102_1
x_Faculty_5_T105_1
x_Faculty_5_T113_1
x_Faculty_5_T207_2
x_Faculty_5_T313_3
x_Faculty_6_T206_2
x_Faculty_6